In [2]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier

In [4]:
import re 
import nltk
from nltk.corpus import stopwords
from nltk.util import pr 
stemmer = nltk.SnowballStemmer("english")
import string
stopwords = set(stopwords.words('english'))


In [5]:
df = pd.read_csv('twitter_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [17]:
df['labels'] = df['class'].map({0:"hate speech ", 1:"offensive language", 2:"neither"})

print(df.head)

<bound method NDFrame.head of                                                    tweet  class  \
0      !!! RT @mayasolovely: As a woman you shouldn't...      2   
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...      1   
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...      1   
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...      1   
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...      1   
...                                                  ...    ...   
24778  you's a muthaf***in lie &#8220;@LifeAsKing: @2...      1   
24779  you've gone and broke the wrong heart baby, an...      2   
24780  young buck wanna eat!!.. dat nigguh like I ain...      1   
24781              youu got wild bitches tellin you lies      1   
24782  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...      2   

                   labels  
0                 neither  
1      offensive language  
2      offensive language  
3      offensive language  
4      offensive language

In [21]:
df = df.dropna()
df = df[['tweet', 'labels']]
df

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,neither
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive language
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,offensive language
24779,"you've gone and broke the wrong heart baby, an...",neither
24780,young buck wanna eat!!.. dat nigguh like I ain...,offensive language
24781,youu got wild bitches tellin you lies,offensive language


In [45]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopwords]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

df['tweet'] = df['tweet'].apply(clean)
df
    

,tweet,labels
0,rt mayasolov woman shouldnt complain clean hou...,neither
1,rt boy dat coldtyga dwn bad cuffin dat hoe place,offensive language
2,rt urkindofbrand dawg rt ever fuck bitch start...,offensive language
3,rt cganderson vivabas look like tranni,offensive language
4,rt shenikarobert shit hear might true might fa...,offensive language
...,...,...
24778,yous muthafin lie coreyemanuel right tl trash ...,offensive language
24779,youv gone broke wrong heart babi drove redneck...,neither
24780,young buck wanna eat dat nigguh like aint fuck...,offensive language
24781,youu got wild bitch tellin lie,offensive language


In [46]:
x = np.array(df['tweet'])
y = np.array(df['labels'])

cv = CountVectorizer()
x = cv.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

DecisionTreeClassifier()

In [47]:
print(dtc.score(x_test, y_test))

0.8754539340954943


In [48]:
def predict(text):
    text = cv.transform([text])
    prediction = dtc.predict(text)
    return prediction[0]

In [57]:
text = "you are great keep up with the good work"

In [58]:
print(predict(text))

neither


In [59]:
text = "you are bad i dont like u"

In [60]:
print(predict(text))

offensive language


In [61]:
text = "you are so stupid"

In [62]:
print(predict(text))

offensive language
